# Cryptocurrency Market Analysis

*Mohammed Jamil*

## Context

The first half of 2022 has been very bad for the crypto market. Crypto industry experts have mixed opinions about the future of cryptocurrencies. While some believe the market will continue to be volatile, others expect some stability in the second half of 2022.

In these uncertain times for the crypto market, is it still worth the risk investing in cryptocurrencies ? and what are the low-risk cryptocurrencies to consider ? These are some questions that we will try to answer in this exploratory analysis and the other notebooks included in this project.

## Table of Contents :

- Data collection.
- Exploratory data analysis.

---

### Data Collection :

First let's import the necessery libraries for this analysis and define some global variables and helper functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import datetime
import time

In [2]:
#Let's define global variables.

current_date = datetime.date.today()

In [3]:
#Let's define some helper functions.

def date_to_posix_timestamp(date):
    """
    Converts datetime.date object or a date string into posix timestamp.
    NB: date string must be in iso format 'YYYY-MM-DD'.
    
    Params:
    -------
        date (datetime.date or string) - Date to convert.
    
    Returns:
    --------
        Posix timestamp (Integer).
    """
    
    if not isinstance(date, datetime.date):
        date = datetime.date.fromisoformat(date)
    
    timestamp = int(time.mktime(date.timetuple()))
    
    return timestamp

To collect cryptocurrency market data, we are going to use the **CoinGecko** API.

Before we start retreiving the historical data about cryptocurrency prices (USD), we need some general information about the currencies supported by the API. 

NB: We will limit this analysis only to the most popular cryptocurrencies.

In [4]:
def get_topN_currencies(n, vs_currency="usd"):
    """
    Retreives the top N cryptocurrencies with the highest market value (N_max = 250).
    
    Params:
    -------
        vs_currency (String) - The target currency of market data.
        n (Integer)          - Number of currencies.
    
    Returns:
    --------
        Top N currencies, their id and symbols (pandas.DataFrame).
    """
    
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {"vs_currency": vs_currency, "order": "market_cap_desc", "per_page": n, "page": 1, "sparkline":False}
    response = requests.get(url, params=params)
    result = np.array(list(map(lambda x: [x['id'], x['symbol'], x['name'], x['market_cap'], x['total_volume']], response.json())))
    data = pd.DataFrame.from_dict({'id': result[:,0],
                                   'symbol': result[:,1],
                                   'name': result[:,2],
                                   'market_cap': result[:,3],
                                   'total_volume': result[:,4]})
    
    return data

In [5]:
currencies_df = get_topN_currencies(100)
print("Number of cryptocurrencies retrieved : ", currencies_df.shape[0])
currencies_df.head()

Number of cryptocurrencies retrieved :  100


,id,symbol,name,market_cap,total_volume
0,bitcoin,btc,Bitcoin,369947615954,30157016176
1,ethereum,eth,Ethereum,197262847971,17777831554
2,tether,usdt,Tether,67630964589,42626579284
3,usd-coin,usdc,USD Coin,51706763129,5794385565
4,binancecoin,bnb,BNB,45582435942,241083759


In [8]:
def coin_historical_data(coin_id, vs_currency='usd', nb_days='max'):
    """
    Fetchs historical data about a specific cryptocurrency from the CoinGecko's API.
    
    Params:
    -------
        coin_sym (String)    - Cryptocurrency id .
        vs_currency (String) - The target currency of market data.
        nb_days (Integer or "max") - Data up to number of days ago (1/7/14/30/90/180/365/max)
    
    Returns:
    --------
        Market historical data of the specified cryptocurrency (pandas.DataFrame).
    
    """
    
    url= "https://api.coingecko.com/api/v3/coins/{}/ohlc".format(coin_id)
    params = {"vs_currency": vs_currency, "days": nb_days}
    response = requests.get(url, params=params)
    result = np.array(response.json())
    data = pd.DataFrame.from_dict({'date': result[:,0],
                                   'open': result[:,1], 
                                   'high': result[:,2], 
                                   'low': result[:,2], 
                                   'close': result[:,4]})
    data.date = data.date.apply(lambda x: datetime.date.fromtimestamp(x/1000))
    data.insert(0, "id", coin_id, True)
    
    return data

Let's check if our function works.

In [12]:
# Let's fetch bitcoin data.

bitcoin_df = coin_historical_data('bitcoin')
print("Number of records : ", bitcoin_df.shape[0])
bitcoin_df.tail()

Number of records :  892


,id,date,open,high,low,close
887,bitcoin,2022-08-23,20945.83,21615.76,21615.76,21387.75
888,bitcoin,2022-08-27,21562.45,21618.01,21618.01,20271.32
889,bitcoin,2022-08-31,20069.95,20308.86,20308.86,19805.35
890,bitcoin,2022-09-03,20023.55,20153.98,20153.98,19941.45
891,bitcoin,2022-09-07,19814.56,19980.01,19980.01,18859.50


Perfect !

---

### Exploratory data analysis :